In [1]:
!pip install scikit-learn scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.4 MB/s eta 0:00:00


In [2]:
#imports
import pandas as pd
import imblearn
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


MEAN IMPUTATION

In [5]:
# reads the mean imputed pre processed data stored on the drive
df = pd.read_csv('/content/drive/MyDrive/JOB-ML/Binary Classification/mean imputation/pre-processed_DF.csv')

In [6]:
# drops the binary label column
df = df.drop('Label', axis=1)

In [40]:
df

,sport,dsport,dur,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
0,47439.0,53.0,0.000004,0,0,0,2.640000e+08,0.000000e+00,2,0,...,0.040453,0.047728,29,29,22,22,22,22,29,Normal
1,35877.0,80.0,1.368633,29,2,370,1.149702e+05,6.350465e+06,370,746,...,0.040453,0.047728,1,3,4,1,1,1,3,Normal
2,37951.0,12319.0,0.002692,29,1,2,7.934621e+05,5.087667e+06,6,8,...,0.040453,0.047728,3,3,5,7,1,1,5,Normal
3,14284.0,40769.0,0.100885,29,1,12,1.371066e+05,2.607563e+06,32,35,...,0.000000,0.000000,8,2,1,4,1,1,1,Normal
4,9167.0,21.0,3.314790,29,11,15,6.945840e+03,8.864514e+03,52,54,...,1.000000,1.000000,1,1,15,12,1,1,5,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430658,47439.0,53.0,0.000001,0,0,0,4.560000e+08,0.000000e+00,2,0,...,0.040453,0.047728,15,15,15,15,15,15,15,Generic
1430659,17293.0,110.0,0.942984,252,5,6,4.470914e+03,5.259898e+03,12,12,...,0.040453,0.047728,2,1,2,4,2,2,2,Exploits
1430660,33654.0,80.0,2.579405,252,103,1,8.330169e+05,3.898573e+03,208,30,...,0.040453,0.047728,2,1,2,4,2,2,2,DoS
1430661,33654.0,80.0,2.579405,252,103,1,8.330169e+05,3.898573e+03,208,30,...,0.040453,0.047728,1,1,2,4,2,2,2,DoS


RandomForest

In [4]:
x = df.iloc[:, 0:-1] # extracts all features
y = df.iloc[:, -1:].values.ravel() # extracts only the last column of attack category

#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
# Define the RF model
model = RandomForestClassifier()

In [22]:
# fit the model using the hyper parameters found during the tuning phase
best_rf_model = RandomForestClassifier(n_estimators=200,
                                       criterion='entropy',
                                       max_features='sqrt',
                                       min_samples_split=5)

# Train the best model using the training data
best_rf_model.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=5,
                       n_estimators=200)

In [23]:
# Make predictions on the test data
predictions = best_rf_model.predict(x_test)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)


# Print the evaluation metrics
print("Accuracy:", accuracy*100)


Accuracy: 97.05905994764672


ANN MODEL

In [7]:
x1 = df.iloc[:, 0:-1]
y1 = df.iloc[:, -1:]

#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

In [15]:
# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the training labels
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()

# Fit and Transform the testing labels
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1)).toarray()


In [16]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fits the scaler on the training data and transforms it
x_train_scaled = scaler.fit_transform(x_train)

# Fits the scaler on the testing data and transforms it
x_test_scaled = scaler.transform(x_test)

In [17]:
# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
x_test_tensor = torch.tensor(x_test_scaled , dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

In [20]:
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [37]:
# Define the dimensions of input, hidden, and output layers
input_size = x_train.shape[1]
hidden_size = 64
output_size = y_train_encoded.shape[1]

# Initialize the ANN model
model = ANN(input_size, hidden_size, output_size)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [38]:
# Set the number of training epochs
num_epochs = 10

# Set the batch size
batch_size = 8

# Set the number of batches
num_batches = len(x_train_tensor) // batch_size

# Training loop
for epoch in range(num_epochs):
    for batch in range(num_batches):
        # Extract the current batch
        start = batch * batch_size
        end = (batch + 1) * batch_size
        inputs = x_train_tensor[start:end]
        labels = y_train_tensor[start:end]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, torch.argmax(labels, dim=1))

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.0767
Epoch [2/10], Loss: 0.0720
Epoch [3/10], Loss: 0.0654
Epoch [4/10], Loss: 0.0614
Epoch [5/10], Loss: 0.0620
Epoch [6/10], Loss: 0.0623
Epoch [7/10], Loss: 0.0586
Epoch [8/10], Loss: 0.0527
Epoch [9/10], Loss: 0.0542
Epoch [10/10], Loss: 0.0524


In [42]:
# Set the model to evaluation mode
model.eval()

# Perform inference on the training set
with torch.no_grad():
    outputs = model(x_train_tensor)

# Get the predicted labels
_, predicted_labels = torch.max(outputs, 1)

# Calculate accuracy
correct = (predicted_labels == torch.argmax(y_train_tensor, dim=1)).sum().item()
total = y_train_tensor.size(0)
accuracy = correct / total
print("Training Accuracy:", accuracy)


Training Accuracy: 0.962778607812814


In [39]:
# Set the model to evaluation mode
model.eval()

# Perform inference on the test set
with torch.no_grad():
    outputs = model(x_test_tensor)

# Get the predicted labels
_, predicted_labels = torch.max(outputs, 1)

# Calculate accuracy
correct = (predicted_labels == torch.argmax(y_test_tensor, dim=1)).sum().item()
total = y_test_tensor.size(0)
accuracy = correct / total
print("Accuracy:", accuracy * 100)



Accuracy: 96.20421272624968


SVM

In [7]:
x2 = df.iloc[:, 0:-1]
y2 = df.iloc[:, -1:].values.ravel()

#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

In [8]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fits the scaler on the training data and transforms it
x_train_scaled = scaler.fit_transform(x_train)

# Fits the scaler on the testing data and transforms it
x_test_scaled = scaler.transform(x_test)

NON-LINEAR SVM

In [9]:
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train_scaled, y_train)

SVC()

In [10]:
y_pred = svm_model.predict(x_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9547937497597271


LINEAR SVM

In [6]:
# Define the SVM model
model = LinearSVC(dual=False)

In [ ]:
# Create the LinearSVC model with the best hyperparameters
best_svm_model = LinearSVC(penalty='l1',tol=2.4004455788496022e-11  ,C=3.9728931339630273,loss='squared_hinge' ,dual=False)
# Train the best model using the training data
best_svm_model.fit(x_train, y_train)

In [9]:
# Make predictions on the test data
predictions = best_svm_model.predict(x_test)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)


# Print the evaluation metrics
print("Accuracy:", accuracy)


Accuracy: 0.9085215616513999


REGRESSION IMPUTATION

RF

In [4]:
# reads the regression imputed pre processed file from the drive
df_regression = pd.read_csv('/content/drive/MyDrive/JOB-ML/Binary Classification/regression imputation/pre-processed_regression_DF.csv')

In [5]:
# drops the label column
df_regression = df_regression.drop('Label', axis=1)

In [5]:
df_regression

,sport,dsport,dur,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
0,47439.0,53.0,0.000004,0.0,0.0,0.0,2.640000e+08,0.000000e+00,2.0,0.0,...,0.416769,0.671370,29.0,29.0,22.0,22.0,22.0,22.0,29.0,Normal
1,35877.0,80.0,1.368633,29.0,2.0,370.0,1.149702e+05,6.350465e+06,370.0,746.0,...,0.538826,0.364746,1.0,3.0,4.0,1.0,1.0,1.0,3.0,Normal
2,37951.0,12319.0,0.002692,29.0,1.0,2.0,7.934621e+05,5.087667e+06,6.0,8.0,...,0.566385,0.436359,3.0,3.0,5.0,7.0,1.0,1.0,5.0,Normal
3,14284.0,40769.0,0.100885,29.0,1.0,12.0,1.371066e+05,2.607563e+06,32.0,35.0,...,0.000000,0.000000,8.0,2.0,1.0,4.0,1.0,1.0,1.0,Normal
4,9167.0,21.0,3.314790,29.0,11.0,15.0,6.945840e+03,8.864514e+03,52.0,54.0,...,1.000000,1.000000,1.0,1.0,15.0,12.0,1.0,1.0,5.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430658,47439.0,53.0,0.000001,0.0,0.0,0.0,4.560000e+08,0.000000e+00,2.0,0.0,...,0.490527,0.535811,15.0,15.0,15.0,15.0,15.0,15.0,15.0,Generic
1430659,17293.0,110.0,0.942984,252.0,5.0,6.0,4.470914e+03,5.259898e+03,12.0,12.0,...,0.589015,0.429452,2.0,1.0,2.0,4.0,2.0,2.0,2.0,Exploits
1430660,33654.0,80.0,2.579405,252.0,103.0,1.0,8.330169e+05,3.898573e+03,208.0,30.0,...,0.467984,0.238979,2.0,1.0,2.0,4.0,2.0,2.0,2.0,DoS
1430661,33654.0,80.0,2.579405,252.0,103.0,1.0,8.330169e+05,3.898573e+03,208.0,30.0,...,0.468905,0.241184,1.0,1.0,2.0,4.0,2.0,2.0,2.0,DoS


In [11]:
x = df_regression.iloc[:, 0:-1]
y = df_regression.iloc[:, -1:].values.ravel()


#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [12]:
#Define the RF model
model = RandomForestClassifier()

In [13]:
# Fit the model on the optimal hyperparameters found during tuning phase
best_rf_model = RandomForestClassifier(n_estimators=200,
                                       criterion='entropy',
                                       max_features='sqrt',
                                       min_samples_split=5)

# Train the best model using the training data
best_rf_model.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=5,
                       n_estimators=200)

In [14]:
# Make predictions on the test data
predictions = best_rf_model.predict(x_test)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)


# Print the evaluation metrics
print("Accuracy:", accuracy*100)

Accuracy: 97.48089175313578


ANN

In [13]:
x2 = df_regression.iloc[:, 0:-1]
y2 = df_regression.iloc[:, -1:]


#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

In [16]:
# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the training labels
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()

# Fit and Transform the testing labels
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1)).toarray()

In [14]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fits the scaler on the training data and transforms it
x_train_scaled = scaler.fit_transform(x_train)

# Fits the scaler on the testing data and transforms it
x_test_scaled = scaler.transform(x_test)

In [18]:
# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
x_test_tensor = torch.tensor(x_test_scaled , dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

In [21]:
# Define the dimensions of input, hidden, and output layers
input_size = x_train.shape[1]
hidden_size = 64
output_size = y_train_encoded.shape[1]

# Initialize the ANN model
model = ANN(input_size, hidden_size, output_size)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [22]:
# Set the number of training epochs
num_epochs = 10

# Set the batch size
batch_size = 8

# Set the number of batches
num_batches = len(x_train_tensor) // batch_size

# Training loop
for epoch in range(num_epochs):
    for batch in range(num_batches):
        # Extract the current batch
        start = batch * batch_size
        end = (batch + 1) * batch_size
        inputs = x_train_tensor[start:end]
        labels = y_train_tensor[start:end]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, torch.argmax(labels, dim=1))

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.0762
Epoch [2/10], Loss: 0.0773
Epoch [3/10], Loss: 0.0280
Epoch [4/10], Loss: 0.0773
Epoch [5/10], Loss: 0.0570
Epoch [6/10], Loss: 0.0909
Epoch [7/10], Loss: 0.0805
Epoch [8/10], Loss: 0.0896
Epoch [9/10], Loss: 0.0826
Epoch [10/10], Loss: 0.0845


In [23]:
# Set the model to evaluation mode
model.eval()

# Perform inference on the test set
with torch.no_grad():
    outputs = model(x_test_tensor)

# Get the predicted labels
_, predicted_labels = torch.max(outputs, 1)

# Calculate accuracy
correct = (predicted_labels == torch.argmax(y_test_tensor, dim=1)).sum().item()
total = y_test_tensor.size(0)
accuracy = correct / total
print("Accuracy:", accuracy * 100)

Accuracy: 96.22657994708754


SVM

In [6]:
x3 = df_regression.iloc[:, 0:-1]
y3 = df_regression.iloc[:, -1:].values.ravel()


#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x3, y3, test_size=0.2, random_state=42)

In [7]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fits the scaler on the training data and transforms it
x_train_scaled = scaler.fit_transform(x_train)

# Fits the scaler on the testing data and transforms it
x_test_scaled = scaler.transform(x_test)

LINEAR SVM

In [10]:
#Define the linear SVC model
model = LinearSVC(dual=False)

In [11]:
# Create the LinearSVC model with the best hyperparameters
best_svm_model = LinearSVC(penalty='l1',tol=2.4004455788496022e-11 ,C=3.9728931339630273,loss='squared_hinge' ,dual=False)
# Train the best model using the training data
best_svm_model.fit(x_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=3.9728931339630273, dual=False, penalty='l1',
          tol=2.4004455788496022e-11)

In [13]:
# Make predictions on the test data
predictions = best_svm_model.predict(x_test_scaled)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)


# Print the evaluation metrics
print("Accuracy:", accuracy)

Accuracy: 0.923696323038587


NON Linear

In [8]:
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train_scaled, y_train)

SVC()

In [9]:
y_pred = svm_model.predict(x_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9553704046719532
